In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sklearn
%pylab inline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as sch
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from itertools import product
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


Загружаем подготовленные данные.

In [2]:
data = pd.read_csv('c6w4-00.csv', sep=';')
data.set_index('index', inplace = True)

data.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
index,,,,,,,,,,,,,,,,,,,,,
2015-06-01 00:00:00,40,32,7,17,60,129,153,271,294,70,...,8,17,10,639,437,230,15,271,90,123
2015-06-01 01:00:00,17,20,5,13,38,90,111,163,246,28,...,6,7,22,547,490,212,21,237,124,119
2015-06-01 02:00:00,5,5,0,8,25,47,97,139,207,25,...,6,11,10,234,215,95,69,253,105,112
2015-06-01 03:00:00,5,6,0,6,27,26,68,77,208,29,...,13,0,2,84,10,17,6,78,3,94
2015-06-01 04:00:00,8,16,2,6,21,39,53,81,109,8,...,20,1,0,9,4,14,5,62,1,1


In [3]:
cl = pd.read_csv('c6w4-cl_ind.csv', sep=';')

cl.head()

,zone,cluster
0,1172,0
1,1173,0
2,1174,0
3,1175,0
4,1182,0


In [4]:
data_cl = []
for i in range(max(cl.cluster)+1):
    data_cl_1 = pd.read_csv('c6w4-5_clust_'+str(i)+'.csv', sep=';')
    data_cl_1.set_index('index', inplace = True)
    data_cl_1.head()
    data_cl.append(data_cl_1)
    

In [5]:
data_cl[2]

,sin1,cos1,sin2,cos2,sin3,cos3,sin4,cos4,sin5,cos5,...,sin46,cos46,sin47,cos47,sin48,cos48,sin49,cos49,sin50,cos50
index,,,,,,,,,,,,,,,,,,,,,
2015-06-01 00:00:00,0.037391,0.999301,0.074730,0.997204,0.111964,0.993712,0.149042,0.988831,0.185912,0.982566,...,0.988831,-0.149042,0.982566,-0.185912,0.974928,-0.222521,9.659258e-01,-0.258819,0.955573,-0.294755
2015-06-01 01:00:00,0.074730,0.997204,0.149042,0.988831,0.222521,0.974928,0.294755,0.955573,0.365341,0.930874,...,-0.294755,-0.955573,-0.365341,-0.930874,-0.433884,-0.900969,-5.000000e-01,-0.866025,-0.563320,-0.826239
2015-06-01 02:00:00,0.111964,0.993712,0.222521,0.974928,0.330279,0.943883,0.433884,0.900969,0.532032,0.846724,...,-0.900969,0.433884,-0.846724,0.532032,-0.781831,0.623490,-7.071068e-01,0.707107,-0.623490,0.781831
2015-06-01 03:00:00,0.149042,0.988831,0.294755,0.955573,0.433884,0.900969,0.563320,0.826239,0.680173,0.733052,...,0.563320,0.826239,0.680173,0.733052,0.781831,0.623490,8.660254e-01,0.500000,0.930874,0.365341
2015-06-01 04:00:00,0.185912,0.982566,0.365341,0.930874,0.532032,0.846724,0.680173,0.733052,0.804598,0.593820,...,0.733052,-0.680173,0.593820,-0.804598,0.433884,-0.900969,2.588190e-01,-0.965926,0.074730,-0.997204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,-0.294755,-0.955573,0.563320,0.826239,-0.781831,-0.623490,0.930874,0.365341,-0.997204,-0.074730,...,0.930874,0.365341,-0.997204,-0.074730,0.974928,-0.222521,-8.660254e-01,0.500000,0.680173,-0.733052
2016-06-30 20:00:00,-0.330279,-0.943883,0.623490,0.781831,-0.846724,-0.532032,0.974928,0.222521,-0.993712,0.111964,...,0.222521,-0.974928,0.111964,0.993712,-0.433884,-0.900969,7.071068e-01,0.707107,-0.900969,-0.433884
2016-06-30 21:00:00,-0.365341,-0.930874,0.680173,0.733052,-0.900969,-0.433884,0.997204,0.074730,-0.955573,0.294755,...,-0.997204,-0.074730,0.955573,-0.294755,-0.781831,0.623490,5.000000e-01,-0.866025,-0.149042,0.988831


1. Для каждой из шести задач прогнозирования \hat{y}_{T+i|T}, i=1,\dots,6 
y
^
​	
  
T+i∣T
​	
 ,i=1,…,6 сформируйте выборки. Откликом будет y_{T+i}y 
T+i
​	
  при всевозможных значениях TT, а признаки можно использовать следующие:
идентификатор географической зоны — категориальный
год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
сами значения прогнозов ARIMA \hat{y}_{T+i|T}^{ARIMA} 
y
^
​	
  
T+i∣T
ARIMA
​	
 
количество поездок из рассматриваемого района в моменты времени y_T, y_{T-1}, \dots, y_{T-K}y 
T
​	
 ,y 
T−1
​	
 ,…,y 
T−K
​	
  (параметр KK можно подбирать; попробуйте начать, например, с 6)
количество поездок из рассматриваемого района в моменты времени y_{T-24}, y_{T-48}, \dots, y_{T-24*K_d}y 
T−24
​	
 ,y 
T−48
​	
 ,…,y 
T−24∗K 
d
​	
 
​	
  (параметр K_dK 
d
​	
  можно подбирать; попробуйте начать, например, с 2)
суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании \hat{y}_{T+i|T}, i=1,\dots,6 
y
^
​	
  
T+i∣T
​	
 ,i=1,…,6 вы можете учитывать только значения yy до момента времени TT включительно.

In [6]:
dat= pd.DataFrame(columns= ['zone', 'tim1', 'hour', 'day', 'week_day', 'week', 'month', 'year', 'val'] + data_cl[0].columns.tolist()
                  , index = range(len(data.index)*len(data.columns)))
#dat.insert(0, ['zone', 'tim1', 'hour', 'day', 'week_day', 'week', 'month', 'year', 'val'])

In [7]:
%%time
k=len(data.index)
for i in range(len(data.columns)):
    dat['zone'] [k*i:k*(1+i)] = data.columns[i]
    dat['tim1'] [k*i:k*(1+i)]= data.index
    dat['val'] [k*i:k*(1+i)]= data.loc[data.index,data.columns[i]]
    
dat['tim1'] = pd.to_datetime(dat['tim1'])
dat['month'] = dat.tim1.map(lambda x: x.month)
dat['hour'] = dat.tim1.map(lambda x: x.hour)
dat['day'] = dat.tim1.map(lambda x: x.day)
dat['week_day'] = dat.tim1.map(lambda x: x.dayofweek)
dat['week'] = dat.tim1.map(lambda x: x.week)
dat['year'] = dat.tim1.map(lambda x: x.year)



Wall time: 47.4 s


In [8]:
%%time
for i in range(len(data.columns)):
    dat.loc[k*i:k*(1+i)-1,'sin1':'cos50']= data_cl[cl.loc[cl.zone == int(data.columns[i]), 'cluster'].item()].set_index([pd.Index(range(k*i,k*(i+1)))])

Wall time: 5min 22s


In [9]:
# Добавляем количество поездок из этого региона за прошлые часы и недели
for i in range(1, 24):
    dat[str(i) + '_hour'] = dat['val'].shift(i)
for i in range(3, 14):
    dat[str(i*12) + '_hour'] = dat['val'].shift(i*12)
for i in range(1, 9):
    dat[str(i) + '_week'] = dat['val'].shift(i*168)

In [10]:
# Добавляем количество поездок для ответов
for i in range(1, 7):
    dat[str(i) + '_ans'] = dat['val'].shift(-i)

In [11]:
#необходимо для работы catboost
dat.loc[:,'sin1':'cos50'] = dat.loc[:,'sin1':'cos50'].astype(float64)
dat.loc[:,'1_hour':] = dat.loc[:,'1_hour':].fillna(0)
dat.loc[:,'1_hour':] =  dat.loc[:,'1_hour':].astype(int64)
dat.loc[:,'zone'] =  dat.loc[:,'zone'].astype(int64)
dat.loc[:,'val'] =  dat.loc[:,'val'].astype(int64)

In [12]:
dat.loc[:,'val':].dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [13]:
dat.loc[:,'val':].dtypes

val        int64
sin1     float64
cos1     float64
sin2     float64
cos2     float64
          ...   
2_ans      int64
3_ans      int64
4_ans      int64
5_ans      int64
6_ans      int64
Length: 149, dtype: object

2. Разбейте каждую из шести выборок на три части:

обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

In [14]:
%%time
train_start = dat['tim1'][8*168]
train_stop = "2016-05-01 00:00:00"
test_stop = "2016-05-31 17:00:00"
pred_start = "2016-05-31 23:00:00"
pred_stop = "2016-06-30 17:00:00"
dat_train = dat[(dat.tim1 >= train_start) & (dat.tim1 <= train_stop)]
dat_test = dat[(dat.tim1 >= train_stop) & (dat.tim1 <= pred_start)]
dat_train_pred = dat[(dat.tim1 >= train_start) & (dat.tim1 <= pred_start)]
dat_pred = dat[(dat.tim1 >= pred_start) & (dat.tim1 <= pred_stop)]

Wall time: 1.95 s


In [15]:
dat.dtypes

zone                 int64
tim1        datetime64[ns]
hour                 int64
day                  int64
week_day             int64
                 ...      
2_ans                int64
3_ans                int64
4_ans                int64
5_ans                int64
6_ans                int64
Length: 157, dtype: object

In [16]:
dat.to_csv('dat_c6w5.csv', sep=';', index=True)

In [17]:
dat

,zone,tim1,hour,day,week_day,week,month,year,val,sin1,...,5_week,6_week,7_week,8_week,1_ans,2_ans,3_ans,4_ans,5_ans,6_ans
0,1075,2015-06-01 00:00:00,0,1,0,23,6,2015,40,0.037391,...,0,0,0,0,17,5,5,8,14,22
1,1075,2015-06-01 01:00:00,1,1,0,23,6,2015,17,0.074730,...,0,0,0,0,5,5,8,14,22,51
2,1075,2015-06-01 02:00:00,2,1,0,23,6,2015,5,0.111964,...,0,0,0,0,5,8,14,22,51,78
3,1075,2015-06-01 03:00:00,3,1,0,23,6,2015,5,0.149042,...,0,0,0,0,8,14,22,51,78,75
4,1075,2015-06-01 04:00:00,4,1,0,23,6,2015,8,0.185912,...,0,0,0,0,14,22,51,78,75,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969403,2168,2016-06-30 19:00:00,19,30,3,26,6,2016,1,-0.294755,...,99,42,58,63,0,1,0,0,0,0
969404,2168,2016-06-30 20:00:00,20,30,3,26,6,2016,0,-0.330279,...,87,113,126,100,1,0,0,0,0,0
969405,2168,2016-06-30 21:00:00,21,30,3,26,6,2016,1,-0.365341,...,84,105,107,107,0,0,0,0,0,0
969406,2168,2016-06-30 22:00:00,22,30,3,26,6,2016,0,-0.399892,...,104,101,127,123,0,0,0,0,0,0


In [18]:
dat.zone.unique()

array([1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132,
       1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182,
       1183, 1184, 1221, 1222, 1223, 1224, 1225, 1227, 1228, 1229, 1230,
       1231, 1232, 1233, 1234, 1235, 1272, 1273, 1274, 1278, 1279, 1280,
       1281, 1282, 1283, 1284, 1285, 1286, 1287, 1326, 1327, 1331, 1332,
       1333, 1334, 1335, 1336, 1337, 1338, 1339, 1376, 1377, 1378, 1380,
       1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1426, 1431,
       1434, 1435, 1436, 1437, 1438, 1439, 1441, 1442, 1480, 1482, 1483,
       1530, 1532, 1533, 1580, 1630, 1684, 1733, 1734, 1783, 2068, 2069,
       2118, 2119, 2168], dtype=int64)

3. Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:

4. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза

A. LinearRegression

In [19]:
%%time
# Создаем мега цикл, который посчитает качество модели и остатки в мае
Koef = []
models_may =dict()
model = LinearRegression(n_jobs=10, normalize=True)
dat_ost= pd.DataFrame(index = dat_test.index)
X_test = dat_test.copy()
X_test.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train = dat_train.copy()
X_train.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
Q_lin = 0

for i in range(1,7):
    Q_may = 0. # ошибка в мае
    Y_test = dat_test[str(i) + '_ans']
    Y_train = dat_train[str(i) + '_ans']
    
    #Считаем ошибку по линейной регрессии
    stud = model.fit(X_train, Y_train)
    Y_pred = stud.predict(X_test)
    Y_pred[Y_pred < 0] = 0  # All low values set to 0
    Y_score = stud.score(X_test, Y_test)
    Koef.append(Y_score)
    models_may[i] = (i, stud)
    Q_may += sum(abs(Y_test-Y_pred))
    Q_lin += Q_may
    dat_ost[str(i) + '_ost'] = Y_test-Y_pred
    print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_may))
    print(' ')
print ('Ошибка линейной модели: '+str(1/(102*739*6.)*Q_lin))

Tочность модели со значением через часа 1 = 0.9691818077768918, ошибка= 3.856253632375023
 
Tочность модели со значением через часа 2 = 0.9402037222806333, ошибка= 5.171026347555967
 
Tочность модели со значением через часа 3 = 0.9226976075360327, ошибка= 5.723639402710472
 
Tочность модели со значением через часа 4 = 0.9103177145843885, ошибка= 6.0505206547577215
 
Tочность модели со значением через часа 5 = 0.90324813249921, ошибка= 6.179367706308471
 
Tочность модели со значением через часа 6 = 0.900959169679491, ошибка= 6.1810850807220294
 
Ошибка линейной модели: 33.161892824429685
Wall time: 2min 1s


B. Ridge

In [ ]:
%%time
# Создаем мега цикл, который посчитает качество модели и остатки в мае, только попробуем другую модель.
from sklearn.model_selection import GridSearchCV

model = Ridge()
Q_rid = 0
X_test = dat_test.copy()
X_test.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train = dat_train.copy()
X_train.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)

for i in range(1,7):
    Q_may = 0. # ошибка в мае
    Y_test = dat_test[str(i) + '_ans']
    Y_train = dat_train[str(i) + '_ans']
    param_grid = [{'alpha' : np.logspace(2, 3, 50)}]
    clf = GridSearchCV(model, param_grid, cv = 5)
    #Считаем ошибку по линейной регрессии
    stud = clf.fit(X_train, Y_train)
    
    
    Y_pred = stud.predict(X_test)
    Y_pred[Y_pred < 0] = 0  # All low values set to 0
    Y_score = stud.score(X_test, Y_test)
    Q_may += sum(abs(Y_test-Y_pred))
    Q_rid += Q_may
    print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_may))
    print(' ')
print ('Ошибка модели кольца: '+str(1/(102*739*6.)*Q_rid))

Tочность модели со значением через часа 1 = 0.9691826707122454, ошибка= 3.855332427100107
 
Tочность модели со значением через часа 2 = 0.9402053552403984, ошибка= 5.169825203224505
 


С. XGB

In [ ]:
import xgboost as xgb

In [33]:
%%time
# Создаем мега цикл, который посчитает качество модели и остатки в мае, только попробуем другую модель.
xgb_params = {'learning_rate': [0.1, 0.3, 0.5], 'n_estimators': [75, 100, 130], 
              'reg_alpha': [0.01, 0.05, 0.1], 'reg_lambda': [0.01, 0.05, 0.1]}
params = list(product(xgb_params['learning_rate'], xgb_params['n_estimators'], 
                      xgb_params['reg_alpha'], xgb_params['reg_lambda']))
X_test = dat_test.copy()
X_test.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train = dat_train.copy()
X_train.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)

best_xgb_cv = None
#best_xgb_score = float('inf')
Q_xgb = 40*102*739*6.
best_xgb_params = None
for j in range(len(params)):
    Q_xgb_1 = 0
    param = params[j]
    xgb_cv = []
    xgb_model = xgb.XGBRegressor(learning_rate=param[0], n_estimators=param[1], 
                                    reg_lambda=param[2], reg_alpha=param[3], n_jobs=-1)
    for i in range(1,7):
        Q_may = 0. # ошибка в мае
        Y_test = dat_test[str(i) + '_ans']
        Y_train = dat_train[str(i) + '_ans']
        
        
        
        #Считаем ошибку по линейной регрессии
        stud = xgb_model.fit(X_train, Y_train)
    
    
        Y_pred = stud.predict(X_test)
        Y_pred[Y_pred < 0] = 0  # All low values set to 0
        Y_score = stud.score(X_test, Y_test)
        Q_may += sum(abs(Y_test-Y_pred))
        Q_xgb_1 += Q_may
        print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_may))
        print(' ')
    
    print ('Ошибка данной модели XGB: '+str(1/(102*739*6.)*Q_xgb_1))
    if Q_xgb_1 < Q_xgb:
        Q_xgb = Q_xgb_1
        best_xgb_cv = xgb_cv
        best_xgb_params = param
print('Лучшие параметры:' +str(best_xgb_params) + ' , ошибка:' + str(1/(102*739*6.)*Q_xgb))
   


[16:32:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 1 = 0.9737415346120699, ошибка= 3.403074615051253
 
[16:36:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 2 = 0.9564669616186323, ошибка= 4.228336959501888
 
[16:39:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 3 = 0.9492657235187073, ошибка= 4.530688109704865
 
[16:44:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 4 = 0.9469511950839903, ошибка= 4.602108981842624
 
[16:47:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 5 = 0.943934

MemoryError: 

In [34]:
print('Лучшие параметры:' +str(best_xgb_params) + ' , ошибка:' + str(1/(102*739*6.)*Q_xgb))

Лучшие параметры:(0.1, 75, 0.01, 0.01) , ошибка:26.20041629078334


Очевидно, что лучший результат у XGB.

5. Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате yyyy.mm.dd, histEndHour — час конца истории от 0 до 23, step \in \{1,2,3,4,5,6\}∈{1,2,3,4,5,6} — номер отсчёта, на который вы делаете предсказание, y — предсказываемое значение.

In [ ]:
%%time
# Создаем мега цикл, который посчитает предсказания для ответа в июне по лучшей моделе
models_june =dict()
answer = []


xgb_model = xgb.XGBRegressor(learning_rate=best_xgb_params[0], n_estimators=best_xgb_params[1], 
                                    reg_lambda=best_xgb_params[2], reg_alpha=best_xgb_params[3], n_jobs=-1)
X_test = dat_pred.copy()
X_test.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train = dat_train_pred.copy()
X_train.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
Q= 0
for i in range(1,7):
    Q_june = 0. # ошибка в июне
    Y_test = dat_pred[str(i) + '_ans']
    Y_train = dat_train_pred[str(i) + '_ans']
    
    #Считаем ошибку по регрессии
    stud = xgb_model.fit(X_train, Y_train)
    Y_pred = stud.predict(X_test)
    Y_score = stud.score(X_test, Y_test)
    
    Y_pred[Y_pred < 0] = 0  # All low values set to 0
    models_june[i] = (i, stud)
    Q_june += sum(abs(Y_test-Y_pred))
    Q+= Q_june
    print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_june))
    for  j in range(len(X_test)):
        answer.append([dat_pred.loc[dat_pred.index[j],'zone'].astype(str)+'_'+dat_pred.loc[dat_pred.index[j],'tim1'].strftime('%Y-%m-%d_%H')+'_'+str(i),int(abs(Y_pred[j]))])
    
    print(' ')
print('Итоговая  ошибка за июнь ='+str(1/(102*739*6.)*Q))

In [ ]:
answer

In [ ]:
answer_1 = pd.DataFrame(answer)

In [ ]:
answer_1.columns = ['id','y']

In [ ]:
answer_1.to_csv('c6w5-ans.csv', sep=',', index=False)

In [35]:
print('Итоговая  ошибка за июнь ='+str(1/(102*739*6.)*Q))

Итоговая  ошибка за июнь =21.786893501198406


На kaggle результат улучшился в 1.5 раза до ошибки 22.48533!!! Это произошло из-за смены модели и добовления значемых признаков.